In [ ]:
import librosa 
import numpy as np 
import os 
import matplotlib.pyplot as plt

In [ ]:
#loading single audio 
audio_path1 = "./weapons audio/andrew1.wav"
y, sr = librosa.load(audio_path1, sr=None)
print("Audio shape:", y.shape)
print("Sample rate:", sr)